**When will the United States reach herd immunity (>230M) for COVID-19?**

Goal: Preict the date of when the first reliable media report is published that states that >230M of the US population (~70%) have either received a SARS-CoV-2 vaccine or has been previously been infected by the virus.

Factors:
* COVID case growth rates
* Vaccines timeline (getting EUA, starting distribution, achieving sufficient scale)
* Delay between achievement and the media article

https://www.metaculus.com/questions/5773/united-states-covid-herd-immunity-230m-date/

In [4]:
import math
import random

import pandas as pd
import numpy as np

from scipy import stats
from datetime import datetime, timedelta


# Model variables
GROWTH_DERIVATIVE_FACTOR_LOW = 5
GROWTH_DERIVATIVE_FACTOR_HIGH = 30

MONTHLY_GROWTH_MODIFIER_LOW = 0.5
MONTHLY_GROWTH_MODIFIER_HIGH = 4.0

DAYS_UNTIL_PEAK_GROWTH_LOW = -30
DAYS_UNTIL_PEAK_GROWTH_HIGH = 60

VACCINE_MODIFIER_LOW = 0.5
VACCINE_MODIFIER_HIGH = 1.5

RANDOM_CASE_FLUCTUATION_LOW = -2000
RANDOM_CASE_FLUCTUATION_HIGH = 4000

DAILY_FOURTH_WAVE_CHANCE = 0.6/365
FOURTH_WAVE_LENGTH_DAYS_LOW = 30
FOURTH_WAVE_LENGTH_DAYS_HIGH = 120

VACCINE_DELAY_DAYS_LOW = 7
VACCINE_DELAY_DAYS_HIGH = 28

NEWS_ARTICLE_DELAY_DAYS_LOW = -10 # Earlier than expected based on estimated cases rather than reported cases
NEWS_ARTICLE_DELAY_DAYS_HIGH = 30

million = 1000000

MONTHLY_VACCINATION_DAILY_AMOUNTS = [np.round(10 * million * (1/30) * (1/12), 0),
                                     np.round(10 * million * (1/30) * (1/12), 0),
                                     np.round(50 * million * (1/30) * (1/12), 0),
                                     np.round(60 * million * (1/30) * (1/12), 0),
                                     np.round(70 * million * (1/30) * (1/12), 0),
                                     np.round(80 * million * (1/30) * (1/12), 0),
                                     np.round(100 * million * (1/30) * (1/12), 0),
                                     np.round(150 * million * (1/30) * (1/12), 0),
                                     np.round(200 * million * (1/30) * (1/12), 0),
                                     np.round(250 * million * (1/30) * (1/12), 0),
                                     np.round(300 * million * (1/30) * (1/12), 0),
                                     np.round(350 * million * (1/30) * (1/12), 0),
                                     np.round(400 * million * (1/30) * (1/12), 0)]

# Simulation parameters
N_SCENARIOS = 1000
SIM_START_DATE = '2020-11-28'
SIM_END_DATE = '2022-12-21'
CURRENT_COVID_CASES = 13.6 * million
CURRENT_COVID_GROWTH = 158000
CURRENT_COVID_DERIVATIVE = 2000
Q4_2020_START_COVID = 6.03 * million
TARGET_POP = 230 * million
US_POP = 328.2 * million

In [5]:
def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '9%'
    elif pct <= 0:
        if digits == 0:
            val = '<1.0%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val


def get_metaculus_range(rangex, minx=None, maxx=None):
    lower25, mean, upper75 = np.percentile(rangex, [25, 50, 75])
    results = {}
    results['lower25'] = lower25
    results['mean'] = mean
    results['upper75'] = upper75
    if minx:
        results['minx'] = print_pct(sum([x < minx for x in rangex]) / len(rangex))
    if maxx:
        results['maxx'] = print_pct(sum([x > maxx for x in rangex]) / len(rangex))
    return results
    

def normal_sample(low, high, interval):
    if (low > high) or (high < low):
        raise ValueError
    if low == high:
        return low
    else:
        mu = (high + low) / 2
        cdf_value = 0.5 + 0.5 * interval
        normed_sigma = stats.norm.ppf(cdf_value)
        sigma = (high - mu) / normed_sigma
        return np.random.normal(mu, sigma)

    
def lognormal_sample(low, high, interval):
    if (low > high) or (high < low):
        raise ValueError
    if low == high:
        return low
    else:
        log_low = np.log(low)
        log_high = np.log(high)
        mu = (log_high + log_low) / 2
        cdf_value = 0.5 + 0.5 * interval
        normed_sigma = stats.norm.ppf(cdf_value)
        sigma = (log_high - mu) / normed_sigma
        return np.random.lognormal(mu, sigma)

In [6]:
days_until_sim_end_date = (datetime.strptime(SIM_END_DATE, '%Y-%m-%d') - datetime.strptime(SIM_START_DATE, '%Y-%m-%d')).days + 1

dec_7_2020_covid_collector = []
y2020_covid_collector = []
q1_2021_covid_collector = []
q2_2021_covid_collector = []
q3_2021_covid_collector = []
q4_2021_covid_collector = []
q1_2021_vaccinated_collector = []
q2_2021_vaccinated_collector = []
q3_2021_vaccinated_collector = []
q4_2021_vaccinated_collector = []
q1_2021_target_collector = []
q2_2021_target_collector = []
q3_2021_target_collector = []
q4_2021_target_collector = []
target_date_collector = []


for n in range(N_SCENARIOS):
    done = False
    fourth_wave = False
    today = datetime.today()
    
    covid = CURRENT_COVID_CASES
    covid_growth = CURRENT_COVID_GROWTH
    
    growth_derivative_factor = np.round(normal_sample(GROWTH_DERIVATIVE_FACTOR_LOW, GROWTH_DERIVATIVE_FACTOR_HIGH, 0.8) + 1, 1)
    if growth_derivative_factor < 0.3:
        growth_derivative_factor = 0.3
    
    monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
    if monthly_growth_modifier < 0.5:
        monthly_growth_modifier = 0.5
     
    peak_growth_days = np.round(normal_sample(DAYS_UNTIL_PEAK_GROWTH_LOW, DAYS_UNTIL_PEAK_GROWTH_HIGH, 0.8), 0)
    if peak_growth_days < 0:
        peak_growth_days = 0
    peak_growth_date = (today + timedelta(days=peak_growth_days)).date()
    
    fourth_wave_days_length = np.round(lognormal_sample(FOURTH_WAVE_LENGTH_DAYS_LOW, FOURTH_WAVE_LENGTH_DAYS_HIGH, 0.8), 0)
    if fourth_wave_days_length < 1:
        fourth_wave_days_length = 1
        
    vaccine_modifier = np.round(normal_sample(VACCINE_MODIFIER_LOW, VACCINE_MODIFIER_HIGH, 0.8), 1)
    if vaccine_modifier < 0.3:
        vaccine_modifier = 0.3
        
    vaccine_delay_days = np.round(normal_sample(VACCINE_DELAY_DAYS_LOW, VACCINE_DELAY_DAYS_HIGH, 0.8), 1)
        
    news_article_delay_days = np.round(normal_sample(NEWS_ARTICLE_DELAY_DAYS_LOW, NEWS_ARTICLE_DELAY_DAYS_HIGH, 0.8), 1)
    
    vaccine_rate = 0
    total_vaccinated = 0
    
    verbose = (n <= 4) or (n % 10 == 0)
    really_verbose = (n <= 4)
    
    if verbose:
        print('-')
        print('-')
        print('## SCENARIO {}/{} ##'.format(n + 1, N_SCENARIOS))
        print('Growth derivative factor: {}'.format(growth_derivative_factor))
        print('Vaccine modifier: {}'.format(vaccine_modifier))
        print('Peak growth on: {}'.format(peak_growth_date))
        print('Vaccine delay days: {}'.format(vaccine_delay_days))
        print('News article delay days: {}'.format(news_article_delay_days))

    for day in range(days_until_sim_end_date):
        date = str((today + timedelta(days = day)).date())
        if really_verbose:
            print('Day {}: {}'.format(day, date))
            
        # Grow vaccines
        days_since_vaccine_start = (today + timedelta(days = day) - datetime(2020, 12, 15) - timedelta(days = vaccine_delay_days)).days
        if days_since_vaccine_start >= 0:
            i = math.floor(days_since_vaccine_start / 30)
            if i >= len(MONTHLY_VACCINATION_DAILY_AMOUNTS):
                i = len(MONTHLY_VACCINATION_DAILY_AMOUNTS) - 1
            daily_vaccines = MONTHLY_VACCINATION_DAILY_AMOUNTS[i]
            old_vaccine_rate = vaccine_rate
            vaccine_rate = np.round(np.random.normal(daily_vaccines * vaccine_modifier,
                                                     daily_vaccines * vaccine_modifier), 0)
            if vaccine_rate < old_vaccine_rate:
                vaccine_rate = old_vaccine_rate
            total_vaccinated = np.round(total_vaccinated + vaccine_rate, 0)
            
        if really_verbose:
            print('...Total vaccinated now {} (+{})'.format(total_vaccinated, vaccine_rate))
            
        if really_verbose:
            print('...Total vaccinated now {} (+{})'.format(total_vaccinated, vaccine_rate))
            
            
        # Grow COVID
        if day > peak_growth_days and (not fourth_wave or day > fourth_wave_end):
            covid_growth -= CURRENT_COVID_DERIVATIVE / growth_derivative_factor * (1 / monthly_growth_modifier)
        else:
            covid_growth += (CURRENT_COVID_DERIVATIVE / growth_derivative_factor) * monthly_growth_modifier
        covid_growth = np.round(covid_growth, 0)
        old_covid_growth = covid_growth
        if old_covid_growth < 0:
            old_covid_growth = 0
            
        covid_growth -= np.round(vaccine_rate * 0.02, 0)
        if covid_growth < 0:
            covid_growth = 0
            
        random_case_fluctuation = np.round(normal_sample(RANDOM_CASE_FLUCTUATION_LOW, RANDOM_CASE_FLUCTUATION_HIGH, 0.8), 0)
        covid += covid_growth + random_case_fluctuation
        current_pop = covid + total_vaccinated
        current_target = current_pop / US_POP * 100
        if current_pop >= TARGET_POP and not done:
            target_date_collector.append(day + news_article_delay_days)
            done = True
            
        if really_verbose:
            print('...Total COVID cases now {} (+{} {} -{} *{})'.format(covid,
                                                                        old_covid_growth,
                                                                        "%+d" % random_case_fluctuation,
                                                                        np.round(vaccine_rate * 0.02, 0),
                                                                        monthly_growth_modifier))
            print('...% of target: {}%/{}%'.format(np.round(current_target, 1),
                                                   np.round(TARGET_POP / US_POP * 100, 1)))
                
        if not fourth_wave and np.random.random() <= DAILY_FOURTH_WAVE_CHANCE:
            if really_verbose:
                print('...COVID FOURTH WAVE!!!')
            fourth_wave = True
            fourth_wave_end = day + fourth_wave_days_length
            
            
        # Record
        if date == '2020-12-07':
            dec7_covid = covid
            if verbose:
                print('...COVID by 7 Dec: {}'.format(dec7_covid))
                    
        if date == '2020-12-31':
            y2020_covid = covid
            if verbose:
                print('...Total 2020 COVID: {}'.format(covid))
                
        if date.split('-')[-1] == '01':
            monthly_growth_modifier = np.round(normal_sample(MONTHLY_GROWTH_MODIFIER_LOW, MONTHLY_GROWTH_MODIFIER_HIGH, 0.8) + 1, 1)
            if monthly_growth_modifier < 0.5:
                monthly_growth_modifier = 0.5
                
        if date == '2021-03-31':
            q1_2021_covid = covid
            q1_2021_vaccinated = total_vaccinated
            q1_2021_target = np.round((covid + total_vaccinated) / US_POP * 100, 1)
            if verbose:
                print('...Q1 2021 COVID: {}'.format(q1_2021_covid))
                print('...Q1 2021 Vaccinated: {}'.format(q1_2021_vaccinated))
                print('...Q1 2021 Target: {}'.format(q1_2021_target))

        if date == '2021-06-30':
            q2_2021_covid = covid
            q2_2021_vaccinated = total_vaccinated
            q2_2021_target = np.round((covid + total_vaccinated) / US_POP * 100, 1)
            if verbose:
                print('...Q2 2021 COVID: {}'.format(q2_2021_covid))
                print('...Q2 2021 Vaccinated: {}'.format(q2_2021_vaccinated))
                print('...Q2 2021 Target: {}'.format(q2_2021_target))
            
        if date == '2021-09-30':
            q3_2021_covid = covid
            q3_2021_vaccinated = total_vaccinated
            q3_2021_target = np.round((covid + total_vaccinated) / US_POP * 100, 1)
            if verbose:
                print('...Q3 2021 COVID: {}'.format(q3_2021_covid))
                print('...Q3 2021 Vaccinated: {}'.format(q3_2021_vaccinated))
                print('...Q3 2021 Target: {}'.format(q3_2021_target))
                
        if date == '2021-12-31':
            q4_2021_covid = covid
            q4_2021_vaccinated = total_vaccinated
            q4_2021_target = np.round((covid + total_vaccinated) / US_POP * 100, 1)
            if verbose:
                print('...Q4 2021 COVID: {}'.format(q4_2021_covid))
                print('...Q4 2021 Vaccinated: {}'.format(q4_2021_vaccinated))
                print('...Q4 2021 Target: {}'.format(q4_2021_target))
   
    if not done:
        target_date_collector.append(1000)

    dec_7_2020_covid_collector.append(dec7_covid)
    y2020_covid_collector.append(y2020_covid)
    q1_2021_covid_collector.append(q1_2021_covid)
    q2_2021_covid_collector.append(q2_2021_covid)
    q3_2021_covid_collector.append(q3_2021_covid)
    q4_2021_covid_collector.append(q4_2021_covid)
    q1_2021_vaccinated_collector.append(q1_2021_vaccinated)
    q2_2021_vaccinated_collector.append(q2_2021_vaccinated)
    q3_2021_vaccinated_collector.append(q3_2021_vaccinated)
    q4_2021_vaccinated_collector.append(q4_2021_vaccinated)
    q1_2021_target_collector.append(q1_2021_target)
    q2_2021_target_collector.append(q2_2021_target)
    q3_2021_target_collector.append(q3_2021_target)
    q4_2021_target_collector.append(q4_2021_target)

-
-
## SCENARIO 1/1000 ##
Growth derivative factor: 17.3
Vaccine modifier: 1.0
Peak growth on: 2020-12-03
Vaccine delay days: 18.1
News article delay days: 22.3
Day 0: 2020-12-03
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 13757173.0 (+158532.0 -1359 -0.0 *4.6)
...% of target: 4.2%/70.1%
Day 1: 2020-12-04
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 13917487.0 (+158507.0 +1807 -0.0 *4.6)
...% of target: 4.2%/70.1%
Day 2: 2020-12-05
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 14079433.0 (+158482.0 +3464 -0.0 *4.6)
...% of target: 4.3%/70.1%
Day 3: 2020-12-06
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 14238460.0 (+158457.0 +570 -0.0 *4.6)
...% of target: 4.3%/70.1%
Day 4: 2020-12-07
...Total vaccinated now 0 (+0)
...Total vaccinated now 0 (+0)
...Total COVID cases now 14400268.0 (+158432.0 +3376 -0.0 *4.6)
...% of targ

...Total vaccinated now 106979292.0 (+1303184.0)
...Total COVID cases now 25658239.0 (+0 +3471 -26064.0 *3.8)
...% of target: 40.4%/70.1%
Day 258: 2021-08-18
...Total vaccinated now 108282476.0 (+1303184.0)
...Total vaccinated now 108282476.0 (+1303184.0)
...Total COVID cases now 25659768.0 (+0 +1529 -26064.0 *3.8)
...% of target: 40.8%/70.1%
Day 259: 2021-08-19
...Total vaccinated now 109585660.0 (+1303184.0)
...Total vaccinated now 109585660.0 (+1303184.0)
...Total COVID cases now 25664789.0 (+0 +5021 -26064.0 *3.8)
...% of target: 41.2%/70.1%
Day 260: 2021-08-20
...Total vaccinated now 110888844.0 (+1303184.0)
...Total vaccinated now 110888844.0 (+1303184.0)
...Total COVID cases now 25668038.0 (+0 +3249 -26064.0 *3.8)
...% of target: 41.6%/70.1%
Day 261: 2021-08-21
...Total vaccinated now 112192028.0 (+1303184.0)
...Total vaccinated now 112192028.0 (+1303184.0)
...Total COVID cases now 25669312.0 (+0 +1274 -26064.0 *3.8)
...% of target: 42.0%/70.1%
Day 262: 2021-08-22
...Total vacci

...Total vaccinated now 619479440.0 (+4228427.0)
...Total vaccinated now 619479440.0 (+4228427.0)
...Total COVID cases now 25822055.0 (+0 +1838 -84569.0 *3.7)
...% of target: 196.6%/70.1%
Day 457: 2022-03-05
...Total vaccinated now 623707867.0 (+4228427.0)
...Total vaccinated now 623707867.0 (+4228427.0)
...Total COVID cases now 25821825.0 (+0 -230 -84569.0 *3.7)
...% of target: 197.9%/70.1%
Day 458: 2022-03-06
...Total vaccinated now 627936294.0 (+4228427.0)
...Total vaccinated now 627936294.0 (+4228427.0)
...Total COVID cases now 25820081.0 (+0 -1744 -84569.0 *3.7)
...% of target: 199.2%/70.1%
Day 459: 2022-03-07
...Total vaccinated now 632164721.0 (+4228427.0)
...Total vaccinated now 632164721.0 (+4228427.0)
...Total COVID cases now 25823970.0 (+0 +3889 -84569.0 *3.7)
...% of target: 200.5%/70.1%
Day 460: 2022-03-08
...Total vaccinated now 636393148.0 (+4228427.0)
...Total vaccinated now 636393148.0 (+4228427.0)
...Total COVID cases now 25821273.0 (+0 -2697 -84569.0 *3.7)
...% of ta

Day 656: 2022-09-20
...Total vaccinated now 1465164840.0 (+4228427.0)
...Total vaccinated now 1465164840.0 (+4228427.0)
...Total COVID cases now 25995521.0 (+0 +4053 -84569.0 *4.4)
...% of target: 454.3%/70.1%
Day 657: 2022-09-21
...Total vaccinated now 1469393267.0 (+4228427.0)
...Total vaccinated now 1469393267.0 (+4228427.0)
...Total COVID cases now 25992069.0 (+0 -3452 -84569.0 *4.4)
...% of target: 455.6%/70.1%
Day 658: 2022-09-22
...Total vaccinated now 1473621694.0 (+4228427.0)
...Total vaccinated now 1473621694.0 (+4228427.0)
...Total COVID cases now 25997094.0 (+0 +5025 -84569.0 *4.4)
...% of target: 456.9%/70.1%
Day 659: 2022-09-23
...Total vaccinated now 1477850121.0 (+4228427.0)
...Total vaccinated now 1477850121.0 (+4228427.0)
...Total COVID cases now 26001861.0 (+0 +4767 -84569.0 *4.4)
...% of target: 458.2%/70.1%
Day 660: 2022-09-24
...Total vaccinated now 1482078548.0 (+4228427.0)
...Total vaccinated now 1482078548.0 (+4228427.0)
...Total COVID cases now 26006300.0 (+0 

...Total vaccinated now 6123213.0 (+115802.0)
...Total COVID cases now 22794551.0 (+48250.0 -326 -2316.0 *3.6)
...% of target: 8.8%/70.1%
Day 76: 2021-02-17
...Total vaccinated now 6239015.0 (+115802.0)
...Total vaccinated now 6239015.0 (+115802.0)
...Total COVID cases now 22838134.0 (+45914.0 -15 -2316.0 *3.6)
...% of target: 8.9%/70.1%
Day 77: 2021-02-18
...Total vaccinated now 6354817.0 (+115802.0)
...Total vaccinated now 6354817.0 (+115802.0)
...Total COVID cases now 22883164.0 (+43578.0 +3768 -2316.0 *3.6)
...% of target: 8.9%/70.1%
Day 78: 2021-02-19
...Total vaccinated now 6470619.0 (+115802.0)
...Total vaccinated now 6470619.0 (+115802.0)
...Total COVID cases now 22926256.0 (+41242.0 +4166 -2316.0 *3.6)
...% of target: 9.0%/70.1%
Day 79: 2021-02-20
...Total vaccinated now 6586421.0 (+115802.0)
...Total vaccinated now 6586421.0 (+115802.0)
...Total COVID cases now 22966395.0 (+38906.0 +3549 -2316.0 *3.6)
...% of target: 9.0%/70.1%
Day 80: 2021-02-21
...Total vaccinated now 67022

...

-
-
## SCENARIO 311/1000 ##
Growth derivative factor: 18.9
Vaccine modifier: 0.7
Peak growth on: 2020-12-03
Vaccine delay days: 19.3
News article delay days: -11.7
...COVID by 7 Dec: 14388804.0
...Total 2020 COVID: 18180719.0
...Q1 2021 COVID: 26574749.0
...Q1 2021 Vaccinated: 11287638.0
...Q1 2021 Target: 11.5
...Q2 2021 COVID: 26700596.0
...Q2 2021 Vaccinated: 54785221.0
...Q2 2021 Target: 24.8
...Q3 2021 COVID: 26783174.0
...Q3 2021 Vaccinated: 131262612.0
...Q3 2021 Target: 48.2
...Q4 2021 COVID: 26864138.0
...Q4 2021 Vaccinated: 286780997.0
...Q4 2021 Target: 95.6
-
-
## SCENARIO 321/1000 ##
Growth derivative factor: 17.5
Vaccine modifier: 1.1
Peak growth on: 2020-12-28
Vaccine delay days: 12.3
News article delay days: 11.5
...COVID by 7 Dec: 14406968.0
...Total 2020 COVID: 18334284.0
...Q1 2021 COVID: 25970711.0
...Q1 2021 Vaccinated: 16670099.0
...Q1 2021 Target: 13.0
...Q2 2021 COVID: 26050277.0
...Q2 2021 Vaccinated: 78207388.0
...Q2 2021 Target: 31.8
...Q3 2021 COVID: 2614622

-
-
## SCENARIO 461/1000 ##
Growth derivative factor: 32.0
Vaccine modifier: 0.6
Peak growth on: 2020-12-16
Vaccine delay days: 20.3
News article delay days: 2.5
...COVID by 7 Dec: 14399565.0
...Total 2020 COVID: 18254635.0
...Q1 2021 COVID: 28585077.0
...Q1 2021 Vaccinated: 6280197.0
...Q1 2021 Target: 10.6
...Q2 2021 COVID: 28778617.0
...Q2 2021 Vaccinated: 37892362.0
...Q2 2021 Target: 20.3
...Q3 2021 COVID: 28898813.0
...Q3 2021 Vaccinated: 107692368.0
...Q3 2021 Target: 41.6
...Q4 2021 COVID: 29018544.0
...Q4 2021 Vaccinated: 261208004.0
...Q4 2021 Target: 88.4
-
-
## SCENARIO 471/1000 ##
Growth derivative factor: 31.2
Vaccine modifier: 0.9
Peak growth on: 2021-01-08
Vaccine delay days: 20.3
News article delay days: 7.8
...COVID by 7 Dec: 14392581.0
...Total 2020 COVID: 18292975.0
...Q1 2021 COVID: 25748775.0
...Q1 2021 Vaccinated: 15836970.0
...Q1 2021 Target: 12.7
...Q2 2021 COVID: 25834738.0
...Q2 2021 Vaccinated: 64220991.0
...Q2 2021 Target: 27.4
...Q3 2021 COVID: 25939045.0


-
-
## SCENARIO 611/1000 ##
Growth derivative factor: 29.8
Vaccine modifier: 0.7
Peak growth on: 2020-12-16
Vaccine delay days: 14.5
News article delay days: 8.8
...COVID by 7 Dec: 14396214.0
...Total 2020 COVID: 18231234.0
...Q1 2021 COVID: 26892494.0
...Q1 2021 Vaccinated: 10623195.0
...Q1 2021 Target: 11.4
...Q2 2021 COVID: 26948981.0
...Q2 2021 Vaccinated: 51563097.0
...Q2 2021 Target: 23.9
...Q3 2021 COVID: 27061138.0
...Q3 2021 Vaccinated: 126436951.0
...Q3 2021 Target: 46.8
...Q4 2021 COVID: 27170733.0
...Q4 2021 Vaccinated: 277068834.0
...Q4 2021 Target: 92.7
-
-
## SCENARIO 621/1000 ##
Growth derivative factor: 27.3
Vaccine modifier: 0.8
Peak growth on: 2020-12-03
Vaccine delay days: 19.6
News article delay days: 13.0
...COVID by 7 Dec: 14392064.0
...Total 2020 COVID: 18194322.0
...Q1 2021 COVID: 25881994.0
...Q1 2021 Vaccinated: 10241399.0
...Q1 2021 Target: 11.0
...Q2 2021 COVID: 25965990.0
...Q2 2021 Vaccinated: 42898264.0
...Q2 2021 Target: 21.0
...Q3 2021 COVID: 26069969.

-
-
## SCENARIO 761/1000 ##
Growth derivative factor: 18.8
Vaccine modifier: 1.2
Peak growth on: 2020-12-03
Vaccine delay days: 19.3
News article delay days: 14.9
...COVID by 7 Dec: 14392356.0
...Total 2020 COVID: 18177448.0
...Q1 2021 COVID: 24294229.0
...Q1 2021 Vaccinated: 18378063.0
...Q1 2021 Target: 13.0
...Q2 2021 COVID: 24333249.0
...Q2 2021 Vaccinated: 82195915.0
...Q2 2021 Target: 32.5
...Q3 2021 COVID: 24411892.0
...Q3 2021 Vaccinated: 219177731.0
...Q3 2021 Target: 74.2
...Q4 2021 COVID: 24507050.0
...Q4 2021 Vaccinated: 464869599.0
...Q4 2021 Target: 149.1
-
-
## SCENARIO 771/1000 ##
Growth derivative factor: 31.3
Vaccine modifier: 1.5
Peak growth on: 2021-01-03
Vaccine delay days: 18.9
News article delay days: 10.5
...COVID by 7 Dec: 14402976.0
...Total 2020 COVID: 18383009.0
...Q1 2021 COVID: 23971624.0
...Q1 2021 Vaccinated: 28844875.0
...Q1 2021 Target: 16.1
...Q2 2021 COVID: 24020023.0
...Q2 2021 Vaccinated: 124323384.0
...Q2 2021 Target: 45.2
...Q3 2021 COVID: 241279

-
-
## SCENARIO 911/1000 ##
Growth derivative factor: 28.4
Vaccine modifier: 1.0
Peak growth on: 2020-12-03
Vaccine delay days: 25.0
News article delay days: 25.3
...COVID by 7 Dec: 14399205.0
...Total 2020 COVID: 18201266.0
...Q1 2021 COVID: 26537074.0
...Q1 2021 Vaccinated: 13966040.0
...Q1 2021 Target: 12.3
...Q2 2021 COVID: 26632825.0
...Q2 2021 Vaccinated: 67532211.0
...Q2 2021 Target: 28.7
...Q3 2021 COVID: 26699905.0
...Q3 2021 Vaccinated: 157066154.0
...Q3 2021 Target: 56.0
...Q4 2021 COVID: 26790174.0
...Q4 2021 Vaccinated: 347003587.0
...Q4 2021 Target: 113.9
-
-
## SCENARIO 921/1000 ##
Growth derivative factor: 29.7
Vaccine modifier: 1.1
Peak growth on: 2020-12-03
Vaccine delay days: 0.9
News article delay days: -10.5
...COVID by 7 Dec: 14399428.0
...Total 2020 COVID: 18045251.0
...Q1 2021 COVID: 22065164.0
...Q1 2021 Vaccinated: 28812023.0
...Q1 2021 Target: 15.5
...Q2 2021 COVID: 22152941.0
...Q2 2021 Vaccinated: 90855090.0
...Q2 2021 Target: 34.4
...Q3 2021 COVID: 2223264

## Total US COVID by 7 Dec

For calibration

In [7]:
get_metaculus_range(dec_7_2020_covid_collector)

{'lower25': 14395230.25, 'mean': 14399386.5, 'upper75': 14404158.75}

## By EOY 2020

In [8]:
get_metaculus_range(y2020_covid_collector)

{'lower25': 18205511.25, 'mean': 18273141.0, 'upper75': 18358849.0}

## By EOQ Q1 2021

In [9]:
get_metaculus_range(q1_2021_covid_collector)

{'lower25': 24514865.5, 'mean': 26178251.5, 'upper75': 28194169.25}

In [10]:
get_metaculus_range(q1_2021_vaccinated_collector)

{'lower25': 10406886.5, 'mean': 15362163.0, 'upper75': 21106628.0}

In [11]:
get_metaculus_range(q1_2021_target_collector)

{'lower25': 11.7, 'mean': 12.7, 'upper75': 14.1}

## By EOQ Q2 2021

In [12]:
get_metaculus_range(q2_2021_covid_collector)

{'lower25': 24622302.75, 'mean': 26299987.0, 'upper75': 28302558.25}

In [13]:
get_metaculus_range(q2_2021_vaccinated_collector)

{'lower25': 49357672.5, 'mean': 68806491.0, 'upper75': 88010906.75}

In [14]:
get_metaculus_range(q2_2021_target_collector)

{'lower25': 23.775000000000002, 'mean': 29.2, 'upper75': 35.025}

## By EOQ Q3 2021

In [15]:
get_metaculus_range(q3_2021_covid_collector)

{'lower25': 24718113.25, 'mean': 26387215.5, 'upper75': 28390592.5}

In [16]:
get_metaculus_range(q3_2021_vaccinated_collector)

{'lower25': 129479285.75, 'mean': 180788805.0, 'upper75': 228102388.0}

In [17]:
get_metaculus_range(q3_2021_target_collector)

{'lower25': 48.27499999999999, 'mean': 63.55, 'upper75': 77.775}

## By EOY 2021

In [18]:
get_metaculus_range(q4_2021_covid_collector)

{'lower25': 24816823.75, 'mean': 26481701.0, 'upper75': 28491810.75}

In [19]:
get_metaculus_range(q4_2021_vaccinated_collector)

{'lower25': 301742673.75, 'mean': 411510157.0, 'upper75': 524654270.75}

In [20]:
get_metaculus_range(q4_2021_target_collector)

{'lower25': 100.975, 'mean': 135.05, 'upper75': 168.47500000000002}

## Target Date

In [21]:
'target not exceeded by 2022-12-21 in {} of scenarios'.format(print_pct(sum([x == days_until_sim_end_date for x in target_date_collector]) / len(target_date_collector)))

'target not exceeded by 2022-12-21 in <1.0% of scenarios'

In [22]:
results = get_metaculus_range([x for x in target_date_collector if x < days_until_sim_end_date])
results['lower25'] = str((datetime.strptime(SIM_START_DATE, '%Y-%m-%d') + timedelta(days=results['lower25'])).date())
results['mean'] = str((datetime.strptime(SIM_START_DATE, '%Y-%m-%d') + timedelta(days=results['mean'])).date())
results['upper75'] = str((datetime.strptime(SIM_START_DATE, '%Y-%m-%d') + timedelta(days=results['upper75'])).date())
results

{'lower25': '2021-09-21', 'mean': '2021-10-17', 'upper75': '2021-11-22'}